In [0]:

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls '/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/'

drive_csv_file.zip	 trip_csv_file.zip  weather_csv_file.zip
drive_vehcile_df_v1.csv  vehicle.csv


### Load Packages

In [4]:
!pip install pygeohash

  Stored in directory: /root/.cache/pip/wheels/3f/5f/14/989d83a271207dda28232746d63e737a2dbd88ea7f7a9db807
Successfully built pygeohash


In [0]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os
from pathlib import Path
import datetime
from datetime import timedelta
from datetime import datetime, timedelta
import pandas as pd
import pygeohash as gh

In [0]:
weather_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/weather_csv_file.zip')

In [7]:
weather_df.head(5)

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit
0,127,39,2017-02-25,11Z,29.9375,-109.0625,278.38,k,0.000,kg/m^2,-1.07,m/s,-0.47,m/s
1,192,51,2017-01-02,18Z,31.4375,-100.9375,289.84,k,0.000,kg/m^2,5.74,m/s,-0.68,m/s
2,254,48,2017-02-23,19Z,31.0625,-93.1875,298.24,k,0.000,kg/m^2,-1.47,m/s,5.61,m/s
3,261,54,2017-01-01,13Z,31.8125,-92.3125,289.25,k,0.010,kg/m^2,-0.86,m/s,1.85,m/s
4,220,49,2017-01-18,10Z,31.1875,-97.4375,278.86,k,0.204,kg/m^2,1.30,m/s,-4.71,m/s


In [0]:
weather_df.shape

In [0]:
weather_df['Temp_F'] = weather_df['temperature_data'].apply(lambda x: (x*(9/5) - 459.67))

In [9]:
weather_df.head(3)

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit,Temp_F
0,127,39,2017-02-25,11Z,29.9375,-109.0625,278.38,k,0.0,kg/m^2,-1.07,m/s,-0.47,m/s,41.414
1,192,51,2017-01-02,18Z,31.4375,-100.9375,289.84,k,0.0,kg/m^2,5.74,m/s,-0.68,m/s,62.042
2,254,48,2017-02-23,19Z,31.0625,-93.1875,298.24,k,0.0,kg/m^2,-1.47,m/s,5.61,m/s,77.162


In [0]:
#Converting to weather condition and weather preciptation categorization
def weather_condition(temp):
  if temp <= 27:
    return 'Snow'
  elif (temp > 27 and temp <= 32):
    return 'Freezing_Rain'
  else:
    return 'Rain'

def weather_precipitation(precep):
  if precep <= 2.5:
    return 'Light'
  elif (precep > 2.5 and precep <= 7.6):
    return 'Moderate'
  else:
    return 'Heavy'

In [0]:
weather_df['weather_condition'] = weather_df['Temp_F'].apply(weather_condition)
weather_df['weather_precipitation'] = weather_df['precipitation_data'].apply(weather_precipitation)

In [14]:
weather_df.head(3)

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit,Temp_F,weather_condition,weather_precipitation
0,127,39,2017-02-25,11Z,29.9375,-109.0625,278.38,k,0.0,kg/m^2,-1.07,m/s,-0.47,m/s,41.414,Rain,Light
1,192,51,2017-01-02,18Z,31.4375,-100.9375,289.84,k,0.0,kg/m^2,5.74,m/s,-0.68,m/s,62.042,Rain,Light
2,254,48,2017-02-23,19Z,31.0625,-93.1875,298.24,k,0.0,kg/m^2,-1.47,m/s,5.61,m/s,77.162,Rain,Light


In [15]:
weather_df.dtypes

x                          int64
y                          int64
date                      object
time                      object
lat                      float64
lon                      float64
temperature_data         float64
temperature_unit          object
precipitation_data       float64
precipitation_unit        object
wind_ew_data             float64
wind_ew_unit              object
wind_ns_data             float64
wind_ns_unit              object
Temp_F                   float64
weather_condition         object
weather_precipitation     object
dtype: object

In [16]:
weather_df['time'].unique()

array(['11Z', '18Z', '19Z', '13Z', '10Z', '07Z', '08Z', '03Z', '06Z',
       '01Z', '05Z', '14Z', '23Z', '00Z', '12Z', '17Z', '21Z', '20Z',
       '15Z', '04Z', '22Z', '09Z', '02Z', '16Z'], dtype=object)

In [17]:
weather_df['weather_condition'].unique()

array(['Rain', 'Freezing_Rain', 'Snow'], dtype=object)

In [18]:
weather_df['weather_precipitation'].unique()

array(['Light', 'Moderate', 'Heavy'], dtype=object)

In [0]:
weather_df['geohash_weather_df']=weather_df.apply(lambda x: gh.encode(x.lat, x.lon,  precision=5), axis=1)

In [20]:
weather_df.head(3)

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit,Temp_F,weather_condition,weather_precipitation,geohash_weather_df
0,127,39,2017-02-25,11Z,29.9375,-109.0625,278.38,k,0.0,kg/m^2,-1.07,m/s,-0.47,m/s,41.414,Rain,Light,9t66q
1,192,51,2017-01-02,18Z,31.4375,-100.9375,289.84,k,0.0,kg/m^2,5.74,m/s,-0.68,m/s,62.042,Rain,Light,9v84z
2,254,48,2017-02-23,19Z,31.0625,-93.1875,298.24,k,0.0,kg/m^2,-1.47,m/s,5.61,m/s,77.162,Rain,Light,9vt8x


In [0]:
#Concatenate date , time and geohash_weather_df - and get a unique ID key
weather_df['Unique_ID'] = weather_df[['date', 'time', 'geohash_weather_df']].apply(lambda x: '_'.join(x), axis = 1)

In [0]:
# get the unique ID count and number of rows


In [22]:
weather_df.head(3)

,x,y,date,time,lat,lon,temperature_data,temperature_unit,precipitation_data,precipitation_unit,wind_ew_data,wind_ew_unit,wind_ns_data,wind_ns_unit,Temp_F,weather_condition,weather_precipitation,geohash_weather_df,Unique_ID
0,127,39,2017-02-25,11Z,29.9375,-109.0625,278.38,k,0.0,kg/m^2,-1.07,m/s,-0.47,m/s,41.414,Rain,Light,9t66q,2017-02-25_11Z_9t66q
1,192,51,2017-01-02,18Z,31.4375,-100.9375,289.84,k,0.0,kg/m^2,5.74,m/s,-0.68,m/s,62.042,Rain,Light,9v84z,2017-01-02_18Z_9v84z
2,254,48,2017-02-23,19Z,31.0625,-93.1875,298.24,k,0.0,kg/m^2,-1.47,m/s,5.61,m/s,77.162,Rain,Light,9vt8x,2017-02-23_19Z_9vt8x


In [0]:
#Concatenate date , time and geohash_weather_df - and get a unique ID key
weather_df['Cond_Precip'] = weather_df[['weather_condition', 'weather_precipitation']].apply(lambda x: '_'.join(x), axis = 1)

In [23]:
weather_df.columns

Index(['x', 'y', 'date', 'time', 'lat', 'lon', 'temperature_data',
       'temperature_unit', 'precipitation_data', 'precipitation_unit',
       'wind_ew_data', 'wind_ew_unit', 'wind_ns_data', 'wind_ns_unit',
       'Temp_F', 'weather_condition', 'weather_precipitation',
       'geohash_weather_df', 'Unique_ID', 'Cond_Precip'],
      dtype='object')

In [0]:
# 'x', 'y', 'temperature_unit', 'precipitation_unit', 'wind_ew_unit', 'wind_ns_unit'
weather_df = weather_df[[ 'Unique_ID', 'Cond_Precip', 'geohash_weather_df']]


### Trip DataFrame

In [0]:
trip_df = pd.read_csv('/content/drive/My Drive/AV_data/Quantum_Black_5th_July_2019/trip_csv_file.zip')

In [28]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity
0,1000516,2e2a9936393a424abf4dea3943c83040,2017-01-01 07:00:00,31.4375,-85.437500,0.00
1,1000516,2e2a9936393a424abf4dea3943c83040,2017-01-01 07:00:01,31.4375,-85.437222,63.41
2,1000516,2e2a9936393a424abf4dea3943c83040,2017-01-01 07:00:02,31.4375,-85.436944,90.37


In [29]:
trip_df[(trip_df['lat'] == 31.0625)].head(4)

,vehicle_id,trip_id,datetime,lat,long,velocity
66987,1000516,356e2c3b278645dd9bf1fbbc88ba5e8c,2017-01-01 19:00:00,31.0625,-88.312500,0.00
66988,1000516,356e2c3b278645dd9bf1fbbc88ba5e8c,2017-01-01 19:00:01,31.0625,-88.312222,40.02
66989,1000516,356e2c3b278645dd9bf1fbbc88ba5e8c,2017-01-01 19:00:02,31.0625,-88.311944,42.50
66990,1000516,356e2c3b278645dd9bf1fbbc88ba5e8c,2017-01-01 19:00:03,31.0625,-88.311667,57.29


In [30]:
trip_df.dtypes

vehicle_id      int64
trip_id        object
datetime       object
lat           float64
long          float64
velocity      float64
dtype: object

In [0]:
from datetime import datetime, time
def convert_datetime(x): 
  return datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S')  

In [0]:
from datetime import datetime
from pytz import timezone


def convert_date_timezone(x):  
  datetime_obj_naive = datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S')
  # Right way!
  datetime_obj_pacific = datetime_obj_naive.astimezone(timezone('US/Pacific'))
  return (datetime_obj_pacific.strftime("%Y-%m-%d %H:%M:%S"))

In [0]:
trip_df['datetime'] =  trip_df['datetime'].apply(lambda x :convert_date_timezone(x)) 

In [34]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity
0,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:00,31.4375,-85.437500,0.00
1,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:01,31.4375,-85.437222,63.41
2,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:02,31.4375,-85.436944,90.37


In [35]:
trip_df.shape

(9217528, 6)

In [36]:
trip_df.dtypes

vehicle_id      int64
trip_id        object
datetime       object
lat           float64
long          float64
velocity      float64
dtype: object

In [0]:
# from datetime import datetime, timedelta
# def reduce_hrs_to_pst(x):
#   date_obj = datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
#   date_obj = date_obj - timedelta(hours=8)
#   return date_obj

In [0]:
# trip_df['datetime_mod_pst'] = trip_df['datetime_mod'].apply(lambda x: reduce_hrs_to_pst(x))

In [0]:
trip_df["date"] = pd.to_datetime(trip_df["datetime"]).dt.date
trip_df["time"] = pd.to_datetime(trip_df["datetime"]).dt.time

In [40]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity,date,time
0,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:00,31.4375,-85.437500,0.00,2016-12-31,23:00:00
1,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:01,31.4375,-85.437222,63.41,2016-12-31,23:00:01
2,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:02,31.4375,-85.436944,90.37,2016-12-31,23:00:02


In [41]:
trip_df.dtypes

vehicle_id      int64
trip_id        object
datetime       object
lat           float64
long          float64
velocity      float64
date           object
time           object
dtype: object

In [0]:
trip_df['geohash_trip_df']=trip_df.apply(lambda x: gh.encode(x.lat, x.long,  precision=5), axis=1)

In [43]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity,date,time,geohash_trip_df
0,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:00,31.4375,-85.437500,0.00,2016-12-31,23:00:00,dje4z
1,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:01,31.4375,-85.437222,63.41,2016-12-31,23:00:01,dje4z
2,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:02,31.4375,-85.436944,90.37,2016-12-31,23:00:02,dje4z


In [0]:
#extract only hour
def get_hour(x):
  return x.hour

trip_df['time_hour'] = trip_df['time'].apply(lambda x: x.hour)

In [45]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity,date,time,geohash_trip_df,time_hour
0,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:00,31.4375,-85.437500,0.00,2016-12-31,23:00:00,dje4z,23
1,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:01,31.4375,-85.437222,63.41,2016-12-31,23:00:01,dje4z,23
2,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:02,31.4375,-85.436944,90.37,2016-12-31,23:00:02,dje4z,23


In [0]:
labels = {
 '00Z' : 17,
 '01Z' : 18,
 '02Z' : 19,
 '03Z' : 20,
 '04Z' : 21,
 '05Z' : 22,
 '06Z' : 23,
 '07Z' : 1,
 '08Z' : 2,
 '09Z' : 3,
 '10Z' : 4,
 '11Z' : 5,
 '12Z' : 6,
 '13Z' : 7,
 '14Z' : 8,
 '15Z' : 9,
 '16Z' : 10,
 '17Z' : 11,
 '18Z' : 12,
 '19Z' : 13,
 '20Z' : 14,
 '21Z' : 15,
 '22Z' : 16,
 '23Z' : 17 
}

In [0]:
labels = dict((v,k) for k,v in labels.items())

In [48]:
labels

{1: '07Z',
 2: '08Z',
 3: '09Z',
 4: '10Z',
 5: '11Z',
 6: '12Z',
 7: '13Z',
 8: '14Z',
 9: '15Z',
 10: '16Z',
 11: '17Z',
 12: '18Z',
 13: '19Z',
 14: '20Z',
 15: '21Z',
 16: '22Z',
 17: '23Z',
 18: '01Z',
 19: '02Z',
 20: '03Z',
 21: '04Z',
 22: '05Z',
 23: '06Z'}

In [0]:
trip_df = trip_df.replace({"time_hour": labels})

In [0]:
trip_df['date'] = trip_df['date'].apply(lambda x: str(x))

In [51]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity,date,time,geohash_trip_df,time_hour
0,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:00,31.4375,-85.437500,0.00,2016-12-31,23:00:00,dje4z,06Z
1,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:01,31.4375,-85.437222,63.41,2016-12-31,23:00:01,dje4z,06Z
2,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:02,31.4375,-85.436944,90.37,2016-12-31,23:00:02,dje4z,06Z


In [52]:
trip_df.dtypes

vehicle_id           int64
trip_id             object
datetime            object
lat                float64
long               float64
velocity           float64
date                object
time                object
geohash_trip_df     object
time_hour           object
dtype: object

In [0]:
trip_df['time_hour'] = trip_df['time_hour'].apply(lambda x: str(x))

In [0]:
#Concatenate date , time and geohash_weather_df - and get a unique ID key
trip_df['Unique_ID'] = trip_df[['date', 'time_hour', 'geohash_trip_df']].apply(lambda x: '_'.join(x), axis = 1)

In [55]:
trip_df.columns

Index(['vehicle_id', 'trip_id', 'datetime', 'lat', 'long', 'velocity', 'date',
       'time', 'geohash_trip_df', 'time_hour', 'Unique_ID'],
      dtype='object')

In [0]:
# Python's program to get start and end of week 
from datetime import datetime, timedelta

def start_end_datetime(x):
  date_obj = datetime.strptime(str(x), '%Y-%m-%d')
  return date_obj - timedelta(days=date_obj.weekday())  # Monday

In [0]:
trip_df['week_start_date'] = trip_df['date'].apply(lambda x:start_end_datetime(x))

In [58]:
trip_df['week_start_date'].unique()

array(['2016-12-26T00:00:00.000000000', '2017-02-20T00:00:00.000000000',
       '2017-02-27T00:00:00.000000000', '2017-01-02T00:00:00.000000000',
       '2017-01-16T00:00:00.000000000', '2017-02-13T00:00:00.000000000',
       '2017-01-09T00:00:00.000000000', '2017-01-30T00:00:00.000000000',
       '2017-01-23T00:00:00.000000000', '2017-02-06T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [59]:
trip_df.head(3)

,vehicle_id,trip_id,datetime,lat,long,velocity,date,time,geohash_trip_df,time_hour,Unique_ID,week_start_date
0,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:00,31.4375,-85.437500,0.00,2016-12-31,23:00:00,dje4z,06Z,2016-12-31_06Z_dje4z,2016-12-26
1,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:01,31.4375,-85.437222,63.41,2016-12-31,23:00:01,dje4z,06Z,2016-12-31_06Z_dje4z,2016-12-26
2,1000516,2e2a9936393a424abf4dea3943c83040,2016-12-31 23:00:02,31.4375,-85.436944,90.37,2016-12-31,23:00:02,dje4z,06Z,2016-12-31_06Z_dje4z,2016-12-26


In [0]:
trip_df_new = trip_df[['vehicle_id', 'week_start_date','datetime','lat', 'long','Unique_ID']]

In [61]:
trip_df_new.shape

(9217528, 6)

In [0]:
del trip_df

In [0]:
# get the important columns from weather data based on unique key ID map
trip_df_new = pd.merge(trip_df_new, weather_df[['Cond_Precip', 'Unique_ID']],on='Unique_ID', how='left')

In [64]:
trip_df_new.shape

(9217528, 7)

In [0]:
del weather_df

In [0]:
# Python 3 program for the  
# haversine formula 
import math 
  
# Python 3 program for the haversine formula 
def haversine(lat1, lon1, lat2, lon2): 
      
    # distance between latitudes 
    # and longitudes 
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
  
    # convert to radians 
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
  
    # apply formulae 
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2)); 
    rad = 6371
    c = 2 * math.asin(math.sqrt(a)) 
    return rad * c 

In [67]:
print(haversine(31.4375, -85.437500,31.4375, -85.437222), "K.M.") 

0.0263745773768242 K.M.


In [68]:
trip_df_new['Cond_Precip'].unique()

array([nan, 'Rain_Light', 'Rain_Moderate', 'Rain_Heavy',
       'Freezing_Rain_Light', 'Snow_Light', 'Freezing_Rain_Moderate',
       'Snow_Moderate'], dtype=object)

In [69]:
trip_df_new[trip_df_new['Cond_Precip'] == 'Rain_Light']['week_start_date'].unique()

array(['2016-12-26T00:00:00.000000000', '2017-02-20T00:00:00.000000000',
       '2017-02-27T00:00:00.000000000', '2017-01-02T00:00:00.000000000',
       '2017-01-16T00:00:00.000000000', '2017-02-13T00:00:00.000000000',
       '2017-01-09T00:00:00.000000000', '2017-01-30T00:00:00.000000000',
       '2017-01-23T00:00:00.000000000', '2017-02-06T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [70]:
trip_df_new.head(5)

,vehicle_id,week_start_date,datetime,lat,long,Unique_ID,Cond_Precip
0,1000516,2016-12-26,2016-12-31 23:00:00,31.4375,-85.437500,2016-12-31_06Z_dje4z,NaN
1,1000516,2016-12-26,2016-12-31 23:00:01,31.4375,-85.437222,2016-12-31_06Z_dje4z,NaN
2,1000516,2016-12-26,2016-12-31 23:00:02,31.4375,-85.436944,2016-12-31_06Z_dje4z,NaN
3,1000516,2016-12-26,2016-12-31 23:00:03,31.4375,-85.436667,2016-12-31_06Z_dje4z,NaN
4,1000516,2016-12-26,2016-12-31 23:00:04,31.4375,-85.436389,2016-12-31_06Z_dje4z,NaN


In [0]:
trip_df_new = trip_df_new.sort_values(['vehicle_id', 'week_start_date', 'datetime'], ascending=[True, True, True])

In [0]:
# Transform the data into lists for each of the columns with an unique index column as trip_id
trip_df_new_lst = trip_df_new.groupby(['vehicle_id', 'week_start_date']).agg(pd.Series.tolist).reset_index()

In [73]:
trip_df_new_lst.head(3)

,vehicle_id,week_start_date,datetime,lat,long,Unique_ID,Cond_Precip
0,1000500,2016-12-26,"[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...","[31.3125, 31.3125, 31.3125, 31.3125, 31.3125, ...","[-85.1875, -85.18722222222223, -85.18694444444...","[2017-01-01_16Z_dje6j, 2017-01-01_16Z_dje6j, 2...","[Rain_Moderate, Rain_Moderate, Rain_Moderate, ..."
1,1000500,2017-01-02,"[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...","[31.8125, 31.8125, 31.8125, 31.8125, 31.8125, ...","[-95.6875, -95.68722222222223, -95.68694444444...","[2017-01-02_11Z_9veuy, 2017-01-02_11Z_9veuy, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh..."
2,1000500,2017-01-09,"[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...","[30.8125, 30.8125, 30.8125, 30.8125, 30.8125, ...","[-84.1875, -84.18722222222223, -84.18694444444...","[2017-01-09_0_djkpk, 2017-01-09_0_djkpk, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [74]:
trip_df_new_lst.shape

(181, 7)

In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  
  
  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  
  for i in range(0,len(RainArr)):    
    if(RainArr[i] == 'Rain_Light'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Rain_Light'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Rain_Light'
trip_df_new_lst['total_light_rain_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [77]:
trip_df_new_lst.head(3)

,vehicle_id,week_start_date,datetime,lat,long,Unique_ID,Cond_Precip,total_light_rain_driving_km
0,1000500,2016-12-26,"[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...","[31.3125, 31.3125, 31.3125, 31.3125, 31.3125, ...","[-85.1875, -85.18722222222223, -85.18694444444...","[2017-01-01_16Z_dje6j, 2017-01-01_16Z_dje6j, 2...","[Rain_Moderate, Rain_Moderate, Rain_Moderate, ...",134.821515
1,1000500,2017-01-02,"[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...","[31.8125, 31.8125, 31.8125, 31.8125, 31.8125, ...","[-95.6875, -95.68722222222223, -95.68694444444...","[2017-01-02_11Z_9veuy, 2017-01-02_11Z_9veuy, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1203.754211
2,1000500,2017-01-09,"[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...","[30.8125, 30.8125, 30.8125, 30.8125, 30.8125, ...","[-84.1875, -84.18722222222223, -84.18694444444...","[2017-01-09_0_djkpk, 2017-01-09_0_djkpk, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",376.334154


In [78]:
trip_df_new_lst['total_light_rain_driving_km'].max()

3153.3405630760167

In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  

  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  for i in range(0,len(RainArr)):
    if(RainArr[i] == 'Freezing_Rain_Light'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Freezing_Rain_Light'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Freezing_Rain_Light'
trip_df_new_lst['total_light_freezing_rain_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [81]:
trip_df_new['Cond_Precip'].unique()

array(['Rain_Moderate', nan, 'Rain_Light', 'Freezing_Rain_Light',
       'Freezing_Rain_Moderate', 'Rain_Heavy', 'Snow_Light',
       'Snow_Moderate'], dtype=object)

In [82]:
trip_df_new_lst.head(4)

,vehicle_id,week_start_date,datetime,lat,long,Unique_ID,Cond_Precip,total_light_rain_driving_km,total_light_freezing_rain_driving_km
0,1000500,2016-12-26,"[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...","[31.3125, 31.3125, 31.3125, 31.3125, 31.3125, ...","[-85.1875, -85.18722222222223, -85.18694444444...","[2017-01-01_16Z_dje6j, 2017-01-01_16Z_dje6j, 2...","[Rain_Moderate, Rain_Moderate, Rain_Moderate, ...",134.821515,0.000000
1,1000500,2017-01-02,"[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...","[31.8125, 31.8125, 31.8125, 31.8125, 31.8125, ...","[-95.6875, -95.68722222222223, -95.68694444444...","[2017-01-02_11Z_9veuy, 2017-01-02_11Z_9veuy, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1203.754211,1285.214182
2,1000500,2017-01-09,"[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...","[30.8125, 30.8125, 30.8125, 30.8125, 30.8125, ...","[-84.1875, -84.18722222222223, -84.18694444444...","[2017-01-09_0_djkpk, 2017-01-09_0_djkpk, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",376.334154,111.346808
3,1000500,2017-01-16,"[2017-01-17 12:00:00, 2017-01-17 12:00:01, 201...","[30.3125, 30.312777777777782, 30.3130555555555...","[-95.6875, -95.6875, -95.6875, -95.6875, -95.6...","[2017-01-17_18Z_9v7uq, 2017-01-17_18Z_9v7uq, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1341.893987,2527.561039


In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  

  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  for i in range(0,len(RainArr)):
    if(RainArr[i] == 'Snow_Light'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Snow_Light'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Snow_Light'
trip_df_new_lst['total_light_snow_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [100]:
trip_df_new_lst.head(10)

,vehicle_id,week_start_date,datetime,lat,long,Unique_ID,Cond_Precip,total_light_rain_driving_km,total_light_freezing_rain_driving_km,total_light_snow_driving_km,total_moderate_rain_driving_km,total_moderate_freezing_rain_driving_km,total_moderate_snow_driving_km,total_heavy_rain_driving_km
0,1000500,2016-12-26,"[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...","[31.3125, 31.3125, 31.3125, 31.3125, 31.3125, ...","[-85.1875, -85.18722222222223, -85.18694444444...","[2017-01-01_16Z_dje6j, 2017-01-01_16Z_dje6j, 2...","[Rain_Moderate, Rain_Moderate, Rain_Moderate, ...",134.821515,0.000000,0.000000,215.000404,0.000000,0.0,0.000000
1,1000500,2017-01-02,"[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...","[31.8125, 31.8125, 31.8125, 31.8125, 31.8125, ...","[-95.6875, -95.68722222222223, -95.68694444444...","[2017-01-02_11Z_9veuy, 2017-01-02_11Z_9veuy, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1203.754211,1285.214182,1683.554514,553.706236,647.281695,0.0,523.917736
2,1000500,2017-01-09,"[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...","[30.8125, 30.8125, 30.8125, 30.8125, 30.8125, ...","[-84.1875, -84.18722222222223, -84.18694444444...","[2017-01-09_0_djkpk, 2017-01-09_0_djkpk, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",376.334154,111.346808,0.000000,0.000000,0.000000,0.0,0.000000
3,1000500,2017-01-16,"[2017-01-17 12:00:00, 2017-01-17 12:00:01, 201...","[30.3125, 30.312777777777782, 30.3130555555555...","[-95.6875, -95.6875, -95.6875, -95.6875, -95.6...","[2017-01-17_18Z_9v7uq, 2017-01-17_18Z_9v7uq, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1341.893987,2527.561039,0.000000,1338.512023,0.000000,0.0,0.000000
4,1000500,2017-01-23,"[2017-01-27 04:00:00, 2017-01-27 04:00:01, 201...","[32.0625, 32.0625, 32.0625, 32.0625, 32.0625, ...","[-100.1875, -100.18722222222223, -100.18694444...","[2017-01-27_10Z_9v8y2, 2017-01-27_10Z_9v8y2, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",439.547780,1090.907509,0.000000,0.000000,0.000000,0.0,0.000000
5,1000500,2017-01-30,"[2017-01-30 00:00:00, 2017-01-30 00:00:01, 201...","[33.1875, 33.18777777777777, 33.18805555555555...","[-95.6875, -95.6875, -95.6875, -95.6875, -95.6...","[2017-01-30_0_9vguy, 2017-01-30_0_9vguy, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",70.874680,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,1000500,2017-02-06,"[2017-02-08 19:00:00, 2017-02-08 19:00:01, 201...","[30.5625, 30.5625, 30.5625, 30.5625, 30.5625, ...","[-91.4375, -91.43722222222223, -91.43694444444...","[2017-02-08_02Z_9vqvz, 2017-02-08_02Z_9vqvz, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",144.550553,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,1000500,2017-02-13,"[2017-02-13 04:00:00, 2017-02-13 04:00:01, 201...","[30.8125, 30.8125, 30.8125, 30.8125, 30.8125, ...","[-110.9375, -110.93722222222223, -110.93694444...","[2017-02-13_10Z_9t3p7, 2017-02-13_10Z_9t3p7, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1326.885732,0.000000,0.000000,311.027038,0.000000,0.0,209.259613
8,1000500,2017-02-20,"[2017-02-20 00:00:00, 2017-02-20 00:00:01, 201...","[30.6875, 30.687777777777782, 30.6880555555555...","[-96.8125, -96.8125, -96.8125, -96.8125, -96.8...","[2017-02-20_0_9v7ns, 2017-02-20_0_9v7ns, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1552.804607,0.000000,0.000000,1664.523945,0.000000,0.0,0.000000
9,1000500,2017-02-27,"[2017-02-27 22:00:00, 2017-02-27 22:00:01, 201...","[30.9375, 30.9375, 30.9375, 30.9375, 30.9375, ...","[-107.6875, -107.68722222222223, -107.68694444...","[2017-02-27_05Z_9t7rv, 2017-02-27_05Z_9t7rv, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",251.056822,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  

  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  for i in range(0,len(RainArr)):
    if(RainArr[i] == 'Rain_Moderate'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Rain_Moderate'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Rain_Moderate'
trip_df_new_lst['total_moderate_rain_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  

  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  for i in range(0,len(RainArr)):
    if(RainArr[i] == 'Freezing_Rain_Moderate'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Freezing_Rain_Moderate'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Freezing_Rain_Moderate'
trip_df_new_lst['total_moderate_freezing_rain_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  

  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  for i in range(0,len(RainArr)):
    if(RainArr[i] == 'Snow_Moderate'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Snow_Moderate'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Snow_Moderate'
trip_df_new_lst['total_moderate_snow_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [0]:
def calculate_distance(RainArr,LatArr, LonArr):  

  n = 0	
  nSpecificWeatherCondition=0
  nTotalKms = 0
  nStartIndexPoint = -1
  flag = 0
  nEndIndexPoint = -1
  
  for i in range(0,len(RainArr)):
    if(RainArr[i] == 'Rain_Heavy'  and flag == 0):      
      nStartIndexPoint = i
      flag = 1
      continue
    elif(flag ==1):
      continue
    elif(RainArr[i] != 'Rain_Heavy'  and flag == 1):
      nEndIndexPoint = i-1
      flag = 0
      nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint],LonArr[nStartIndexPoint],LatArr[nEndIndexPoint], LonArr[nEndIndexPoint])
           
  if(flag == 1):
    nTotalKms = nTotalKms + haversine(LatArr[nStartIndexPoint], LonArr[nStartIndexPoint], LatArr[n-1], LonArr[n-1])
  return nTotalKms     
      


In [0]:
nSpecificWeatherCondition = 'Rain_Heavy'
trip_df_new_lst['total_heavy_rain_driving_km']  = trip_df_new_lst.apply(lambda row: calculate_distance(row['Cond_Precip'], row['lat'], row['long']), axis=1)

In [94]:
trip_df_new_lst.head(5)

,vehicle_id,week_start_date,datetime,lat,long,Unique_ID,Cond_Precip,total_light_rain_driving_km,total_light_freezing_rain_driving_km,total_light_snow_driving_km,total_moderate_rain_driving_km,total_moderate_freezing_rain_driving_km,total_moderate_snow_driving_km,total_heavy_rain_driving_km
0,1000500,2016-12-26,"[2017-01-01 10:00:00, 2017-01-01 10:00:01, 201...","[31.3125, 31.3125, 31.3125, 31.3125, 31.3125, ...","[-85.1875, -85.18722222222223, -85.18694444444...","[2017-01-01_16Z_dje6j, 2017-01-01_16Z_dje6j, 2...","[Rain_Moderate, Rain_Moderate, Rain_Moderate, ...",134.821515,0.000000,0.000000,215.000404,0.000000,0.0,0.000000
1,1000500,2017-01-02,"[2017-01-02 05:00:00, 2017-01-02 05:00:01, 201...","[31.8125, 31.8125, 31.8125, 31.8125, 31.8125, ...","[-95.6875, -95.68722222222223, -95.68694444444...","[2017-01-02_11Z_9veuy, 2017-01-02_11Z_9veuy, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1203.754211,1285.214182,1683.554514,553.706236,647.281695,0.0,523.917736
2,1000500,2017-01-09,"[2017-01-09 00:00:00, 2017-01-09 00:00:01, 201...","[30.8125, 30.8125, 30.8125, 30.8125, 30.8125, ...","[-84.1875, -84.18722222222223, -84.18694444444...","[2017-01-09_0_djkpk, 2017-01-09_0_djkpk, 2017-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",376.334154,111.346808,0.000000,0.000000,0.000000,0.0,0.000000
3,1000500,2017-01-16,"[2017-01-17 12:00:00, 2017-01-17 12:00:01, 201...","[30.3125, 30.312777777777782, 30.3130555555555...","[-95.6875, -95.6875, -95.6875, -95.6875, -95.6...","[2017-01-17_18Z_9v7uq, 2017-01-17_18Z_9v7uq, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",1341.893987,2527.561039,0.000000,1338.512023,0.000000,0.0,0.000000
4,1000500,2017-01-23,"[2017-01-27 04:00:00, 2017-01-27 04:00:01, 201...","[32.0625, 32.0625, 32.0625, 32.0625, 32.0625, ...","[-100.1875, -100.18722222222223, -100.18694444...","[2017-01-27_10Z_9v8y2, 2017-01-27_10Z_9v8y2, 2...","[Rain_Light, Rain_Light, Rain_Light, Rain_Ligh...",439.547780,1090.907509,0.000000,0.000000,0.000000,0.0,0.000000


In [95]:
trip_df_new_lst.isnull().sum()

vehicle_id                                 0
week_start_date                            0
datetime                                   0
lat                                        0
long                                       0
Unique_ID                                  0
Cond_Precip                                0
total_light_rain_driving_km                0
total_light_freezing_rain_driving_km       0
total_light_snow_driving_km                0
total_moderate_rain_driving_km             0
total_moderate_freezing_rain_driving_km    0
total_moderate_snow_driving_km             0
total_heavy_rain_driving_km                0
dtype: int64

In [0]:
weather_features = trip_df_new_lst.drop(['datetime','lat', 'long', 'Unique_ID','Cond_Precip'],axis=1)

In [0]:
weather_features = weather_features.sort_values(['vehicle_id', 'week_start_date'], ascending=[True, True])

In [0]:
weather_features.to_csv('weather_features.csv', index=False)